In [ ]:
import os
import pickle
import shapely
import numpy as np
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from datetime import date
import geonetworkx as gnx
from networkx.classes.filters import no_filter
from statistics import mode
from tqdm import tqdm
from shapely import LineString

tqdm.pandas()

SHARED_PROJECT_PATH = '...'
SHARED_PROJECT_PATH_POLY = '...'

In [ ]:
# Get OSM raod network within Central Hungary with necessary attributes without merging road segments on specific attributes
useful_tags = ['surface', 'lit', 'lanes:forward', 'lanes:backward']
ox.settings.useful_tags_way += useful_tags
graph = ox.graph_from_place('Central Hungary', simplify=False, retain_all=True,
                            custom_filter = '["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|residential"]')
graph = ox.simplification.simplify_graph(graph, edge_attrs_differ=['highway', 'maxspeed'] + useful_tags)

In [ ]:
# Add geometry columns to non-simplified graph edges
for k in graph.edges:
    if 'geometry' not in graph.edges[k].keys():
        graph.edges[k]['geometry'] = LineString([[graph.nodes[k[0]]['x'], graph.nodes[k[0]]['y']], [graph.nodes[k[1]]['x'], graph.nodes[k[1]]['y']]])

In [ ]:
# Discretize roads to linestrings
graph = gnx.read_geograph_with_coordinates_attributes(graph)
graph_view = nx.graphviews.subgraph_view(graph, filter_node=no_filter, filter_edge=no_filter)
edges_as_lines = nx.get_edge_attributes(graph_view, graph.edges_geometry_key)
gdf_graph_edges = gpd.GeoDataFrame({'gnx_edge_id': list(edges_as_lines.keys()), 'geometry': list(edges_as_lines.values())}, crs='EPSG:4326')
print(gdf_graph_edges.shape)

In [ ]:
# Read Budapest polygons
gdf_poly = gpd.read_file(os.path.join(SHARED_PROJECT_PATH_POLY, 'bp_polygons_osm.geojson'))

In [ ]:
# Get Budapest road network by overlaying the Budapest polygon on the Central Hungary road network
gdf_graph_edges = gdf_graph_edges.overlay(gdf_poly.iloc[0:1], how='intersection', keep_geom_type=True)
print(gdf_graph_edges.shape)

In [ ]:
# Add relevant graph attributes to road network gdf
filter_gnx_dict = lambda d, keys: {k: d[k] if k in d.keys() else np.nan for k in keys}
attributes = ['highway', 'maxspeed', 'oneway', 'length', 'lit', 'surface']
s = gdf_graph_edges.apply(lambda r: filter_gnx_dict(graph_view.edges[r.gnx_edge_id], attributes), axis=1)
gdf_graph_edges = pd.concat([gdf_graph_edges, pd.DataFrame(s.to_list())], axis=1)
print(gdf_graph_edges.shape)

In [ ]:
# Drop redundant columns containing a single value
gdf_graph_edges.drop(columns=['name'], inplace=True)
print(gdf_graph_edges.shape)

In [ ]:
# Solve problem of two-way road not having the same gdf_edge_id for both directions - 1 example found in the Budapest road network for now
twoway_dir_set = set(gdf_graph_edges[~gdf_graph_edges['oneway']]['gnx_edge_id'].to_list())
twoway_undir_set = set(gdf_graph_edges[~gdf_graph_edges['oneway']]['gnx_edge_id'].apply(lambda t: t if t[0] < t[1] else (t[1], t[0], t[2])).to_list())
twoway_undir_dupl_set = twoway_undir_set.union(set(map(lambda t: (t[1], t[0], t[2]), twoway_undir_set)))
twoway_undir_dupl_set.difference(twoway_dir_set)

In [ ]:
gdf_graph_edges[gdf_graph_edges['gnx_edge_id'].apply(lambda t: (t[0] == 1380502054) | (t[1] == 1380502054))]

In [ ]:
gdf_graph_edges.loc[gdf_graph_edges['gnx_edge_id'] == (1380502040, 1380502054, 0), 'gnx_edge_id'] = 'temp'
filt = gdf_graph_edges['gnx_edge_id'] == (1380502040, 1380502054, 1)
gdf_graph_edges.loc[filt, 'gnx_edge_id'] = pd.Series([(1380502040, 1380502054, 0)], index=filt[filt].index)
filt = gdf_graph_edges['gnx_edge_id'] == 'temp'
gdf_graph_edges.loc[filt, 'gnx_edge_id'] = pd.Series([(1380502040, 1380502054, 1)], index=filt[filt].index)

In [ ]:
# Remove 0 maxspeeds
for k in gdf_graph_edges[gdf_graph_edges['maxspeed'] == '0'].gnx_edge_id:
    del graph.edges[k]['maxspeed']
gdf_graph_edges.loc[gdf_graph_edges[gdf_graph_edges['maxspeed'] == '0'].index, 'maxspeed'] = np.nan
print(gdf_graph_edges.shape)

In [ ]:
# Aggregate surface attributes
paved_attrs = ['chipseal', 'cobblestone', 'compacted', 'concrete', 'sett', 'paving_stones']
unpaved_attrs = ['dirt', 'earth', 'fine_gravel', 'grass', 'gravel', 'ground', 'pebblestone', 'rock', 'sand']

def replace_graph_attributes(graph, attr, old, new):
    for k in graph.edges:
        if attr in graph.edges[k].keys() and graph.edges[k][attr] == old:
            graph.edges[k][attr] = new
    return graph

for k in gdf_graph_edges[~(gdf_graph_edges['surface'].isna()) & (gdf_graph_edges['surface'].str.contains('cobblestone'))].gnx_edge_id:
    graph.edges[k]['surface'] = 'cobblestone'
for k in gdf_graph_edges[~(gdf_graph_edges['surface'].isna()) & (gdf_graph_edges['surface'].str.contains('concrete'))].gnx_edge_id:
    graph.edges[k]['surface'] = 'concrete'
for k in gdf_graph_edges[~(gdf_graph_edges['surface'].isna()) & (gdf_graph_edges['surface'].str.contains('grass'))].gnx_edge_id:
    graph.edges[k]['surface'] = 'grass'
for k in gdf_graph_edges[gdf_graph_edges['surface'].isin(paved_attrs)].gnx_edge_id:
    graph.edges[k]['surface'] = 'paved'
for k in gdf_graph_edges[gdf_graph_edges['surface'].isin(unpaved_attrs)].gnx_edge_id:
    graph.edges[k]['surface'] = 'unpaved'
for old in paved_attrs:
    graph = replace_graph_attributes(graph, 'surface', old, 'paved')
for old in unpaved_attrs:
    graph = replace_graph_attributes(graph, 'surface', old, 'unpaved')

gdf_graph_edges.loc[gdf_graph_edges[~(gdf_graph_edges['surface'].isna()) & (gdf_graph_edges['surface'].str.contains('cobblestone'))].index, 'surface'] = 'cobblestone'
gdf_graph_edges.loc[gdf_graph_edges[~(gdf_graph_edges['surface'].isna()) & (gdf_graph_edges['surface'].str.contains('concrete'))].index, 'surface'] = 'concrete'
gdf_graph_edges.loc[gdf_graph_edges[~(gdf_graph_edges['surface'].isna()) & (gdf_graph_edges['surface'].str.contains('grass'))].index, 'surface'] = 'grass'
gdf_graph_edges.loc[gdf_graph_edges[gdf_graph_edges['surface'].isin(['chipseal', 'cobblestone', 'compacted', 'concrete', 'sett', 'paving_stones'])].index, 'surface'] = 'paved'
gdf_graph_edges.loc[gdf_graph_edges[gdf_graph_edges['surface'].isin(['dirt', 'earth', 'fine_gravel', 'grass', 'gravel', 'ground', 'pebblestone', 'rock', 'sand'])].index, 'surface'] = 'unpaved'

print(gdf_graph_edges.shape)

In [ ]:
# Find missing attributes for an edge as the most common value of the 1-hop neighbors of the same type
def find_attr_from_neighbors(graph, edge, attr, dt):
    o, d, _ = edge

    l = (list(nx.bfs_edges(graph, source=o, depth_limit=dt)) + list(nx.bfs_edges(graph, source=d, depth_limit=dt)))
    if (o, d) in l:
        l.remove((o, d))
    if (d, o) in l:
        l.remove((d, o))
    
    neighb_attrs = []
    for it in l:
        i = 0
        while not it + (i,) in graph.edges.keys():
            i += 1
        e = graph.edges[it + (i,)]
        if attr in e.keys() and e['highway'] == graph.edges[edge]['highway']:
            neighb_attrs.append(e[attr])
    if len(neighb_attrs):
        return mode(neighb_attrs)
    return None


for attr in ['maxspeed', 'lit', 'surface']:
    print(attr)
    print('NaNs before:', gdf_graph_edges[gdf_graph_edges[attr].isna()].shape[0])
    for dt in tqdm([1, 1, 1, 1, 1, 3, 3, 3, 5, 5, 5, 20, 20, 20, 1000]):
        for i in gdf_graph_edges[gdf_graph_edges[attr].isna()].index:
            edge = gdf_graph_edges.loc[i, 'gnx_edge_id']
            val = find_attr_from_neighbors(graph, edge, attr, dt)
            if val != None:
                gdf_graph_edges.loc[i, attr] = val
                graph.edges[edge][attr] = val
    print('NaNs after:', gdf_graph_edges[gdf_graph_edges[attr].isna()].shape[0])
    print()
print(gdf_graph_edges.shape)

In [ ]:
# Complete missing attributes with the most common value from all road segments of the same type
for attr in ['maxspeed', 'lit', 'surface']:
    print(attr)
    print('NaNs before:', gdf_graph_edges[gdf_graph_edges[attr].isna()].shape[0])
    common_attr_per_highway = gdf_graph_edges.groupby(['highway'])[attr].agg(pd.Series.mode)
    gdf_graph_edges.loc[gdf_graph_edges[gdf_graph_edges[attr].isna()].index, attr] = gdf_graph_edges[gdf_graph_edges[attr].isna()].apply(
                                                                                         lambda row: common_attr_per_highway[row['highway']], axis=1)
    print('NaNs after:', gdf_graph_edges[gdf_graph_edges[attr].isna()].shape[0])
    print()
print(gdf_graph_edges.shape)

In [ ]:
# Extract link information to a sepparate column
gdf_graph_edges['link'] = False
gdf_graph_edges.loc[gdf_graph_edges[gdf_graph_edges['highway'].str.contains('_')].index, 'link'] = True
gdf_graph_edges.loc[:, 'highway'] = gdf_graph_edges.highway.str.split('_').apply(lambda l: l[0])
print(gdf_graph_edges.shape)

In [ ]:
# Add geographical information as coordinates of linestring representative point
gdf_graph_edges['repr_lon'] = gdf_graph_edges['geometry'].to_crs('EPSG:23700').representative_point().x
gdf_graph_edges['repr_lat'] = gdf_graph_edges['geometry'].to_crs('EPSG:23700').representative_point().y
print(gdf_graph_edges.shape)

In [ ]:
# Convert CUT directed road network gdf back to graph

def node_name_logic(edge_ls, graph, eps=1e-3):
    from_node_no = int(edge_ls['gnx_edge_id'][0])
    if np.abs(graph.nodes[from_node_no]['x'] - shapely.get_point(edge_ls.geometry, 0).x) > eps or \
        np.abs(graph.nodes[from_node_no]['y'] - shapely.get_point(edge_ls.geometry, 0).y) > eps:
        # introduce new node id if non-existent in OSM road network
        from_node_no = f'CUT_{edge_ls.name}_{from_node_no}'
    edge_ls['FROMNODENO'] = from_node_no

    to_node_no = int(edge_ls['gnx_edge_id'][1])
    if np.abs(graph.nodes[to_node_no]['x'] - shapely.get_point(edge_ls.geometry, -1).x) > eps or \
        np.abs(graph.nodes[to_node_no]['y'] - shapely.get_point(edge_ls.geometry, -1).y) > eps:
        # introduce new node id if non-existent in OSM road network
        to_node_no = f'CUT_{edge_ls.name}_{to_node_no}'
    edge_ls['TONODENO'] = to_node_no

    edge_ls['KEY'] = int(edge_ls['gnx_edge_id'][2])

    return edge_ls


def derive_node_gdf_from_osm_lines(edges):
    edges["geom1"] = shapely.get_point(edges.geometry, 0)
    edges["geom2"] = shapely.get_point(edges.geometry, -1)

    fromnode = edges.groupby(['FROMNODENO','geom1']).size().reset_index()
    tonode = edges.groupby(['TONODENO','geom2']).size().reset_index()

    nodes = pd.DataFrame(np.concatenate((fromnode.values, tonode.values), axis=0), columns=['osmid', 'geometry', 'size'])
    nodes = gpd.GeoDataFrame(nodes.drop_duplicates(subset='osmid'), geometry='geometry')
    nodes.drop(columns=['size'], inplace=True)   # it is ambiguous as calculated above - size_in / size_out would be correct 

    nodes['x'] = nodes.geometry.x
    nodes['y'] = nodes.geometry.y
    
    return nodes.set_index('osmid')


def create_edge_gdf_from_osm_lines(edges, attribute_cols):
    edges['u'] = edges['FROMNODENO']
    edges['v'] = edges['TONODENO']
    edges['key'] = edges['KEY']

    edges = edges[['u', 'v', 'key', 'geometry'] + attribute_cols]
    return edges.set_index(['u', 'v', 'key'])


def replace_node_ids_with_value(edge_id, to_replace, value):
    edge_id = list(edge_id)
    if edge_id[0] == to_replace:
        edge_id[0] = value
    if edge_id[1] == to_replace:
        edge_id[1] = value
    return tuple(edge_id)


# Explode MultiLineStrings resulted by overlay to LineStrings
gdf_graph_edges = gdf_graph_edges.explode(index_parts=False)
gdf_graph_edges.reset_index(inplace=True, drop=True)

attribute_cols = gdf_graph_edges.columns[1:-1].to_list()

gdf_bp_edges = gdf_graph_edges.progress_apply(lambda row: node_name_logic(row, graph), axis=1)
gdf_graph_edges['gnx_edge_id'] = list(zip(gdf_bp_edges['FROMNODENO'], gdf_bp_edges['TONODENO'], gdf_bp_edges['KEY']))

gdf_nodes = derive_node_gdf_from_osm_lines(gdf_bp_edges)
gdf_edges = create_edge_gdf_from_osm_lines(gdf_bp_edges, attribute_cols=attribute_cols)

# Drop redundant nodes obtained by CUTTING - nodes with same coordinates, but different ids
df = gdf_nodes[gdf_nodes.index.get_level_values('osmid').astype(str).str.contains('CUT')]
redundant_node_list = list(filter(lambda t: len(t)>1, df.groupby(list(df))['geometry'].apply(lambda x: tuple(x.index)).tolist()))
for group in tqdm(redundant_node_list):
    for node in group[1:]:
        gdf_nodes.drop(node, inplace=True)
        gdf_edges.index = list(map(lambda edge_id: replace_node_ids_with_value(edge_id, node, group[0]), gdf_edges.index.tolist()))
        gdf_graph_edges['gnx_edge_id'] = gdf_graph_edges['gnx_edge_id'].apply(lambda edge_id: replace_node_ids_with_value(edge_id, node, group[0]))

# Increase key - 3rd tuple component - of duplicated edge ids
gdf_edges = gdf_edges.reset_index()
while gdf_edges[gdf_edges['index'].duplicated()].shape[0]:
    gdf_edges.loc[gdf_edges[gdf_edges['index'].duplicated()].index, 'index'] = gdf_edges.loc[gdf_edges[gdf_edges['index'].duplicated()].index, 'index'].apply(lambda row: (row[0], row[1], row[2]+1))
gdf_edges.set_index('index', inplace=True)
while gdf_graph_edges[gdf_graph_edges['gnx_edge_id'].duplicated()].shape[0]:
    gdf_graph_edges.loc[gdf_graph_edges[gdf_graph_edges['gnx_edge_id'].duplicated()].index, 'gnx_edge_id'] = gdf_graph_edges.loc[gdf_graph_edges[gdf_graph_edges['gnx_edge_id'].duplicated()].index, 'gnx_edge_id'].apply(lambda row: (row[0], row[1], row[2]+1))

graph_bp_edges = ox.convert.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=None)

print(gdf_graph_edges.shape)
print(gdf_bp_edges.shape)
print(len(graph_bp_edges.edges))

In [ ]:
# Change order of FROM- and TONODE elements of gnx_edge_id for oneway streets that have been changed in the graph by the to_undirected
def change_edge_id_by_graph_node_order(row):
    if row[0] in list(graph_bp_edges.nodes) and row[1] in list(graph_bp_edges.nodes) and\
                list(graph_bp_edges.nodes).index(row[0]) > list(graph_bp_edges.nodes).index(row[1]):
        return (row[1], row[0], row[2])
    return row
gdf_graph_edges.loc[gdf_graph_edges[gdf_graph_edges['oneway']].index, 'gnx_edge_id'] = gdf_graph_edges[gdf_graph_edges['oneway']]['gnx_edge_id'].progress_apply(change_edge_id_by_graph_node_order)

In [ ]:
# Keep only the largest fully connected graph from the road network
graph_u = ox.convert.to_undirected(graph_bp_edges)
S = [graph_u.subgraph(c).copy() for c in nx.connected_components(graph_u)]
max_comp = S[np.argmax([len(g.edges) for g in S])]
max_comp_edges = list(max_comp.edges)
gdf_graph_edges_whole_undir = gdf_graph_edges[gdf_graph_edges['gnx_edge_id'].isin(list(graph_u.edges))].copy()
gdf_graph_edges_conn_undir = gdf_graph_edges[gdf_graph_edges['gnx_edge_id'].isin(max_comp_edges)].copy()
print(len(graph_u.edges))
print(gdf_graph_edges_whole_undir.shape)
print(len(max_comp.edges))
print(gdf_graph_edges_conn_undir.shape)

In [ ]:
# Convert gdfs to graph once more to get rid of remaining anomalies
df = gdf_graph_edges_conn_undir.copy()
df['FROMNODENO'] = df['gnx_edge_id'].apply(lambda t: t[0])
df['TONODENO'] = df['gnx_edge_id'].apply(lambda t: t[1])
df['KEY'] = df['gnx_edge_id'].apply(lambda t: t[2])
gdf_nodes = derive_node_gdf_from_osm_lines(df)
gdf_edges = create_edge_gdf_from_osm_lines(df, attribute_cols=attribute_cols)
graph_conn = ox.convert.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=None)
gdf_graph_edges_conn_undir = ox.convert.graph_to_gdfs(graph_conn, nodes=False)
gdf_graph_edges_conn_undir.reset_index(inplace=True)
gdf_graph_edges_conn_undir['gnx_edge_id'] = gdf_graph_edges_conn_undir.apply(lambda row: (row['u'], row['v'], row['key']), axis=1)
gdf_graph_edges_conn_undir.drop(columns=['u', 'v', 'key', 'osmid'], inplace=True)

df = gdf_graph_edges_whole_undir.copy()
df['FROMNODENO'] = df['gnx_edge_id'].apply(lambda t: t[0])
df['TONODENO'] = df['gnx_edge_id'].apply(lambda t: t[1])
df['KEY'] = df['gnx_edge_id'].apply(lambda t: t[2])
gdf_nodes = derive_node_gdf_from_osm_lines(df)
gdf_edges = create_edge_gdf_from_osm_lines(df, attribute_cols=attribute_cols)
graph_whole = ox.convert.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=None)
gdf_graph_edges_whole_undir = ox.convert.graph_to_gdfs(graph_whole, nodes=False)
gdf_graph_edges_whole_undir.reset_index(inplace=True)
gdf_graph_edges_whole_undir['gnx_edge_id'] = gdf_graph_edges_whole_undir.apply(lambda row: (row['u'], row['v'], row['key']), axis=1)
gdf_graph_edges_whole_undir.drop(columns=['u', 'v', 'key', 'osmid'], inplace=True)

print(len(graph_conn.edges))
print(gdf_graph_edges_conn_undir.shape)
print(len(graph_whole.edges))
print(gdf_graph_edges_whole_undir.shape)

In [ ]:
# Get largest fully connected graph as directed gnx graph
max_comp_edges_dir = max_comp_edges + [(tup[1], tup[0], tup[2]) for tup in max_comp_edges]  # Add swapped indexes for directed graph
gdf_graph_edges_dir = gdf_graph_edges[gdf_graph_edges['gnx_edge_id'].isin(max_comp_edges_dir)].copy()

gdf_graph_edges_dir['FROMNODENO'] = gdf_graph_edges_dir['gnx_edge_id'].apply(lambda x: x[0])
gdf_graph_edges_dir['TONODENO'] = gdf_graph_edges_dir['gnx_edge_id'].apply(lambda x: x[1])
gdf_graph_edges_dir['KEY'] = gdf_graph_edges_dir['gnx_edge_id'].apply(lambda x: x[2])

gdf_nodes = derive_node_gdf_from_osm_lines(gdf_graph_edges_dir)
gdf_edges = create_edge_gdf_from_osm_lines(gdf_graph_edges_dir, attribute_cols=attribute_cols)

graph_bp_edges_dir = ox.convert.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=None)

gdf_dir = ox.convert.graph_to_gdfs(graph_bp_edges_dir, nodes=False)

print(len(graph_bp_edges_dir.edges))
print(gdf_dir.shape)

In [ ]:
# Save processed largest connected component road network as geojson
gdf_graph_edges_conn_undir['gnx_edge_id'] = gdf_graph_edges_conn_undir['gnx_edge_id'].astype(str)
today = date.today().strftime('%d-%m-%Y')
out_path = os.path.join(SHARED_PROJECT_PATH, 'data', 'OSM_road_network', today)
if not os.path.exists(out_path):
    os.makedirs(out_path)
gdf_graph_edges_conn_undir.to_file(os.path.join(out_path, f'GDF_BP-network-conn-undir_{today}.json'), driver='GeoJSON')

In [ ]:
# Save processed largest connected component road network as geojson
gdf_graph_edges_whole_undir['gnx_edge_id'] = gdf_graph_edges_whole_undir['gnx_edge_id'].astype(str)
gdf_graph_edges_whole_undir.to_file(os.path.join(out_path, f'GDF_BP-network-whole-undir_{today}.json'), driver='GeoJSON')

In [ ]:
# Save processed largest connected component road network as directed gnx graph
pickle.dump(graph_bp_edges_dir, open(os.path.join(out_path, f'GNX_BP-network-conn-dir_{today}.json'), 'wb'))

In [ ]:
# Save processed largest connected component road network as directed gnx graph
pickle.dump(graph_bp_edges, open(os.path.join(out_path, f'GNX_BP-network-whole-dir_{today}.json'), 'wb'))